In [1]:
import pickle
import sys
import os
import datetime

import sklearn
import pandas as pd
import numpy as np
from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt

In [51]:
#Import data from the processing of data in the journal_training_word2vec notebook
df = pickle.load(open('/mnt/mypartition/pubmed_nlp_research/C_article_replication_and_translational_classification/mixed_journals_article/df.p', 'rb'))

In [54]:
#testing just on article
df = pickle.load(open('/mnt/mypartition/pubmed_nlp_research/C_article_replication_and_translational_classification/df.p', 'rb'))

In [62]:
#testing just on journals
df = pickle.load(open('/mnt/mypartition/pubmed_nlp_research/C_article_replication_and_translational_classification/pickled_objects/dfJournals_trans_categories.p', 'rb'))
val_art = pickle.load(open('/mnt/mypartition/pubmed_nlp_research/C_article_replication_and_translational_classification/df.p', 'rb'))
df.columns = ['label', 'title', 'abstract', 'qualifier_terms', 'descriptor_terms']

In [52]:
val_art = pickle.load(open('/mnt/mypartition/pubmed_nlp_research/C_article_replication_and_translational_classification/mixed_journals_article/val_art.p', 'rb'))
val_journals = pickle.load(open('/mnt/mypartition/pubmed_nlp_research/C_article_replication_and_translational_classification/mixed_journals_article/val_journals.p', 'rb'))

In [55]:
print df.shape
print df.columns.values
print df['label'].unique()

(354, 6)
['pmid' 'title' 'abstract' 'qualifier_terms' 'descriptor_terms' 'label']
['T0' 'T1/T2' 'T3/T4' 'TX']


In [40]:
df[0:5]

,title,abstract,qualifier_terms,descriptor_terms,label,boolean_label,dataset
54,Pulmonary hypertension and other potentially f...,Systemic juvenile idiopathic arthritis (JIA) i...,complications diagnosis mortality diagnosis et...,"Adolescent Arthritis, Juvenile Child Child, Pr...",T0,1,article
270,Are increased weight and appetite useful indic...,"During childhood and adolescence, physiologica...",physiology diagnosis physiopathology physiology,"Adolescent Appetite Child Child, Preschool Dep...",T0,1,article
191,How physician electronic health record screen ...,Use of electronic health records (EHRs) in pri...,,,T3/T4,0,article
213,Early identification and treatment of Alzheime...,Alzheimer's disease (AD) is a progressive neur...,diagnosis economics psychology therapy economi...,Alzheimer Disease Caregivers Cost Savings Cost...,T3/T4,0,article
320,Caregiver person-centeredness and behavioral s...,Mealtime behavioral symptoms are distressing a...,,,T3/T4,0,article


In [41]:
val_journals[0:5]

,title,abstract,qualifier_terms,descriptor_terms,label,boolean_label
31401,Loss of lysosomal ion channel transient recept...,Mucolipidosis type IV (MLIV) is a lysosomal st...,genetics metabolism genetics metabolism pathol...,Cathepsin B Cytoplasm HeLa Cells Humans Lysoso...,T0,1
73091,Bendamustine plus rituximab versus fludarabine...,Fludarabine-based chemoimmunotherapy with ritu...,adverse effects therapeutic use administration...,Aged Antineoplastic Combined Chemotherapy Prot...,T3/T4,0
22852,Oligonucleotide recombination enabled site-spe...,Recombineering refers to a strategy for engine...,genetics metabolism methods chemistry genetics...,"Bacteria Mutagenesis, Site-Directed Oligonucle...",T0,1
70755,A Comparative biochemical study on two marine ...,Two marine endophytic bacteria were isolated f...,pharmacology pharmacology growth & development...,Anti-Infective Agents Antioxidants Bacillus En...,T3/T4,0
42100,High therapeutic potential of positive alloste...,There are currently no clinically efficacious ...,pharmacology drug effects pathology physiology...,Animals Anti-Inflammatory Agents Astrocytes Br...,T1/T2,0


In [42]:
val_art[0:5]

,title,abstract,qualifier_terms,descriptor_terms,label,boolean_label
59,Where is the action? Action sentence processin...,According to an influential view of conceptual...,diagnosis etiology physiology complications ph...,"Aged Aged, 80 and over Analysis of Variance Co...",T0,1
329,Frontal gray matter reduction after breast can...,Cognitive changes related to cancer and its tr...,adverse effects therapeutic use pathology drug...,Adult Aged Antineoplastic Combined Chemotherap...,T3/T4,0
237,Deriving competencies for mentors of clinical ...,Although the importance of research mentorship...,education education,"Career Mobility Communication Ethics, Research...",TX,0
178,Long-term satisfaction and predictors of use o...,Intracorporeal injections have low use rates a...,drug therapy etiology statistics & numerical d...,Erectile Dysfunction Humans Injections Male Mi...,T3/T4,0
135,Association of 9-hydroxy risperidone concentra...,Risperidone has been used to treat behavioral ...,drug therapy adverse effects pharmacokinetics ...,"Aged Aged, 80 and over Alzheimer Disease Antip...",T1/T2,0


In [63]:
a = datetime.datetime.now().replace(microsecond=0)

y = df['label'].values
skf = StratifiedKFold(y, n_folds=5, shuffle=True)

#include mesh terms in bag of words?
mesh_terms = True
#remove stopwords?
stops = False
#if we want to set max features or not for the BOWs
maxF = 5000
#stem words?
stem = False

j = 1
avgT0 = []
avgT1T2 = []
avgT3T4 = []
val_scores_journals_T0 = []
val_scores_journals_T1T2 = []
val_scores_journals_T3T4 = []
val_scores_article_T0 = []
val_scores_article_T1T2 = []
val_scores_article_T3T4 = []


for train_index, test_index in skf:
    #get first test/train split and use
    if j == 1:

        print("ROUND", j)

        #use the indexes to subset the df pandas dataframe
        train1, test1 = df.iloc[train_index], df.iloc[test_index]

        print "Sizes of train and test sets:"
        print len(train1)
        print len(test1)

        print "Cleaning and parsing the training set abstracts...\n"
        # Initialize an empty list to hold the clean reviews
        clean_train_reviews = []

        # Loop over each review; create an index i that goes from 0 to the length
        # of the pandas df column
        for i in range(train1["abstract"].size):
            # Call our function for each one, and add the result to the list of
            # clean reviews
            words = KaggleWord2VecUtility.review_to_words( train1["abstract"].iloc[i], remove_stopwords=stops, stemmer=stem)
            words = words + ' ' + KaggleWord2VecUtility.review_to_words( train1["title"].iloc[i], remove_stopwords=stops, stemmer=stem)
            if mesh_terms:
                words = words + ' ' + KaggleWord2VecUtility.review_to_words( train1["descriptor_terms"].iloc[i], remove_stopwords=stops, stemmer=stem)
                words = words + ' ' + KaggleWord2VecUtility.review_to_words( train1["qualifier_terms"].iloc[i], remove_stopwords=stops, stemmer=stem)
            clean_train_reviews.append(words)

        print len(clean_train_reviews)

        #with tfidf word weighting
        #uses L2 norm by default
        print "Creating the bag of words...\n"

        # Initialize the "TfidfVectorizer" object, which is scikit-learn's
        # bag of words tool.  
        vectorizer = TfidfVectorizer(analyzer = "word",   \
                                 tokenizer = None,    \
                                 preprocessor = None, \
                                #already removed stopwords when preprocessing reviews into words
                                 stop_words = None,
                                max_features = maxF)

        # fit_transform() does two functions: First, it fits the model
        # and learns the vocabulary; second, it transforms our training data
        # into feature vectors. The input to fit_transform should be a list of 
        # strings.
        train_data_features = vectorizer.fit_transform(clean_train_reviews)
        print len(vectorizer.get_feature_names())

        # Numpy arrays are easy to work with, so convert the result to an 
        # array
        train_data_features = train_data_features.toarray()
        print train_data_features.shape

        #Moving on to test set...
        # Create an empty list and append the clean reviews one by one
        clean_test_reviews = [] 

        print "Cleaning and parsing the test set abstracts...\n"
        for i in range(test1["abstract"].size):    
            words = KaggleWord2VecUtility.review_to_words( test1["abstract"].iloc[i], remove_stopwords=stops, stemmer=stem)
            words = words + ' ' + KaggleWord2VecUtility.review_to_words( test1["title"].iloc[i], remove_stopwords=stops, stemmer=stem)
            if mesh_terms:
                words = words + ' ' + KaggleWord2VecUtility.review_to_words( test1["descriptor_terms"].iloc[i], remove_stopwords=stops, stemmer=stem)
                words = words + ' ' + KaggleWord2VecUtility.review_to_words( test1["qualifier_terms"].iloc[i], remove_stopwords=stops, stemmer=stem)
            clean_test_reviews.append(words)

        print "Adding tfidf weights and converting to Bag of Words..."
        # Get a bag of words for the test set, and convert to a numpy array
        test_data_features = vectorizer.transform(clean_test_reviews)
        test_data_features = test_data_features.toarray()

        #------------------------------------------------------------------------------------------------------------    
        print "Training the random forest..."

        #binarize labels
        x = label_binarize(train1['label'], classes=['T0', 'T1/T2', 'T3/T4'])
        n_classes = x.shape[1]
        print n_classes

        x_test = label_binarize(test1['label'], classes=['T0', 'T1/T2', 'T3/T4'])

        # Initialize a Random Forest classifier with 100 trees
        forest = OneVsRestClassifier(RandomForestClassifier(n_estimators = 100, max_depth=1000, n_jobs=-1))

        # Fit the forest to the training set, using the bag of words as 
        # features and the sentiment labels as the response variable
        #
        # This may take a few minutes to run
        forest = forest.fit( train_data_features, x )

        #get the feature importance for each of the three fits
        print len(forest.estimators_)
        feats = []
        for i in range(len(forest.estimators_)):
            feats.append(forest.estimators_[i].feature_importances_)

        print "Using the random forest to make sentiment label predictions..."
        result = forest.predict(test_data_features)

        print "Scoring the test set"

        # Compute ROC curve and ROC area for each class
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(x_test[:, i], result[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])

        print roc_auc

        avgT0.append(roc_auc[0])
        avgT1T2.append(roc_auc[1])
        avgT3T4.append(roc_auc[2])

        #---------------------------------------------------------------------------------------------------------------
        #Moving on to validation set...
        # Create an empty list and append the clean reviews one by one
        clean_test_reviews_val_art = [] 

        print "Cleaning and parsing validation set 1's abstracts...\n"
        for i in range(val_art["abstract"].size):    
            words = KaggleWord2VecUtility.review_to_words( val_art["abstract"].iloc[i], remove_stopwords=stops)
            words = words + ' ' + KaggleWord2VecUtility.review_to_words( val_art["title"].iloc[i], remove_stopwords=stops)
            if mesh_terms:
                words = words + ' ' + KaggleWord2VecUtility.review_to_words( val_art["descriptor_terms"].iloc[i], remove_stopwords=stops)
                words = words + ' ' + KaggleWord2VecUtility.review_to_words( val_art["qualifier_terms"].iloc[i], remove_stopwords=stops)
            clean_test_reviews_val_art.append(words)

        # Get a bag of words for the validation set, and convert to a numpy array
        test_data_features_val_art = vectorizer.transform(clean_test_reviews_val_art)
        test_data_features_val_art = test_data_features_val_art.toarray()

    #     #repeat for second validation set
    #     clean_test_reviews_val_journals = [] 

    #     print "Cleaning and parsing validation set 2's abstracts...\n"
    #     for i in range(val_journals["abstract"].size):    
    #         words = KaggleWord2VecUtility.review_to_words( val_journals["abstract"].iloc[i], remove_stopwords=stops)
    #         words = words + ' ' + KaggleWord2VecUtility.review_to_words( val_journals["title"].iloc[i], remove_stopwords=stops)
    #         if mesh_terms:
    #             words = words + ' ' + KaggleWord2VecUtility.review_to_words( val_journals["descriptor_terms"].iloc[i], remove_stopwords=stops)
    #             words = words + ' ' + KaggleWord2VecUtility.review_to_words( val_journals["qualifier_terms"].iloc[i], remove_stopwords=stops)
    #         clean_test_reviews_val_journals.append(words)

    #     # Get a bag of words for the validation set, and convert to a numpy array
    #     test_data_features_val_journals = vectorizer.transform(clean_test_reviews_val_journals)
    #     test_data_features_val_journals = test_data_features_val_journals.toarray()

        print "Scoring validation set 1..."
        # Use the random forest to make sentiment label predictions
        #using forest fitted on training data
        result_val = forest.predict(test_data_features_val_art)

        x_test_val = label_binarize(val_art['label'], classes=['T0', 'T1/T2', 'T3/T4'])

        # Compute ROC curve and ROC area for each class
        fpr_val = dict()
        tpr_val = dict()
        roc_auc_val = dict()
        for i in range(n_classes):
            fpr_val[i], tpr_val[i], _ = roc_curve(x_test_val[:, i], result_val[:, i])
            roc_auc_val[i] = auc(fpr_val[i], tpr_val[i])

        print roc_auc_val
        val_scores_article_T0.append(roc_auc_val[0])
        val_scores_article_T1T2.append(roc_auc_val[1])
        val_scores_article_T3T4.append(roc_auc_val[2])  

    #     print "Scoring validation set 2..."
    #     # Use the random forest to make sentiment label predictions
    #     #using forest fitted on training data
    #     result_val = forest.predict(test_data_features_val_journals)

    #     x_test_val = label_binarize(val_journals['label'], classes=['T0', 'T1/T2', 'T3/T4'])

    #     # Compute ROC curve and ROC area for each class
    #     fpr_val = dict()
    #     tpr_val = dict()
    #     roc_auc_val = dict()
    #     for i in range(n_classes):
    #         fpr_val[i], tpr_val[i], _ = roc_curve(x_test_val[:, i], result_val[:, i])
    #         roc_auc_val[i] = auc(fpr_val[i], tpr_val[i])

    #     print roc_auc_val
    #     val_scores_journals_T0.append(roc_auc_val[0])
    #     val_scores_journals_T1T2.append(roc_auc_val[1])
    #     val_scores_journals_T3T4.append(roc_auc_val[2])

        #----------------------------------------------------------------------------------------------------------------
        #Testing makeup of training and testing between journals and article dataframes

    #     print ""
    #     print 'Makeup of training and test sets'
    #     print len(train1.loc[train1['dataset'] == 'journal'])/ float(len(train1))
    #     print len(test1.loc[test1['dataset'] == 'journal']) / float(len(test1))
        print "------------------------------------------------------------------"
        print ""
    j = j + 1
    
        
print "Average scores on test set: "
print ('T0: ', sum(avgT0) / float(len(avgT0)))
print ('T1/T2: ', sum(avgT1T2) / float(len(avgT1T2)))
print ('T3/T4: ', sum(avgT3T4) / float(len(avgT3T4)))

print "Average scores on article validation set: "
print ('T0: ', sum(val_scores_article_T0) / float(len(val_scores_article_T0)))
print ('T1/T2: ', sum(val_scores_article_T1T2) / float(len(val_scores_article_T1T2)))
print ('T3/T4: ', sum(val_scores_article_T3T4) / float(len(val_scores_article_T3T4)))

# print "Average scores on journals validation set: "
# print ('T0: ', sum(val_scores_journals_T0) / float(len(val_scores_journals_T0)))
# print ('T1/T2: ', sum(val_scores_journals_T1T2) / float(len(val_scores_journals_T1T2)))
# print ('T3/T4: ', sum(val_scores_journals_T3T4) / float(len(val_scores_journals_T3T4)))

b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

('ROUND', 1)
Sizes of train and test sets:
77637
19412
Cleaning and parsing the training set abstracts...

77637
Creating the bag of words...

5000
(77637, 5000)
Cleaning and parsing the test set abstracts...

Adding tfidf weights and converting to Bag of Words...
Training the random forest...
3
3
Using the random forest to make sentiment label predictions...
Scoring the test set
{0: 0.9752431797421196, 1: 0.90251974542318292, 2: 0.90582853409396247}
Cleaning and parsing validation set 1's abstracts...

Scoring validation set 1...
{0: 0.60995370370370361, 1: 0.73915055532702589, 2: 0.70597381342062193}
------------------------------------------------------------------

Average scores on test set: 
('T0: ', 0.9752431797421196)
('T1/T2: ', 0.90251974542318292)
('T3/T4: ', 0.90582853409396247)
Average scores on article validation set: 
('T0: ', 0.60995370370370361)
('T1/T2: ', 0.73915055532702589)
('T3/T4: ', 0.70597381342062193)
0:08:27


In [13]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords

from nltk import PorterStemmer

class KaggleWord2VecUtility(object):
    """KaggleWord2VecUtility is a utility class for processing raw HTML text into segments for further learning"""

    @staticmethod
    def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text()
        #
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        #
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]
        #
        # 5. Return a list of words
        return(words)
    
    @staticmethod
    def review_to_words( review, remove_stopwords=False, stemmer=False ):
        # Function to convert a raw review to a string of words
        # The input is a single string (a raw movie review), and 
        # the output is a single string (a preprocessed movie review)
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text() 
        #
        # 2. Remove non-letters        
        review_text = re.sub("[^a-zA-Z]", " ", review_text) 
        #
        # 3. Convert to lower case, split into individual words
        words = review_text.lower().split()                             
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]   
            
        if stemmer:
            words = [PorterStemmer().stem_word(w) for w in words]
        #
        # 6. Join the words back into one string separated by space, 
        # and return the result.
        return( " ".join( words ))   

    # Define a function to split a review into parsed sentences
    @staticmethod
    def review_to_sentences( review, tokenizer, remove_stopwords=False ):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        #
        # 1. Use the NLTK tokenizer to split the paragraph into sentences
        raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
        #
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append( KaggleWord2VecUtility.review_to_wordlist( raw_sentence, \
                  remove_stopwords ))
        #
        # Return the list of sentences (each sentence is a list of words,
        # so this returns a list of lists
        return sentences